In [1]:
%matplotlib inline
import pymongo
from pymongo import MongoClient
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
import matplotlib.pyplot as plt
import numpy
from pprint import pprint
from collections import defaultdict
from datetime import datetime, timedelta
from functools import partial
import time
db = MongoClient('localhost').condor

In [110]:
avg = 0.
num = 0
for row in db.condor_history.find({'EnteredCurrentStatus':{'$gte':time.mktime((2016,1,1,0,0,0,0,0,0)),
                                       '$lt':time.mktime((2017,1,1,0,0,0,0,0,0))},
                                   'LastRemoteHost':{'$regex':'chtc'},
                                  },
                                  projection=['RemoteWallClockTime']):
    avg += row['RemoteWallClockTime']
    num += 1
print('total',avg/3600)

total 7019583.989444445


In [108]:
avg = 0.
num = 0
for row in db.condor_history.find({'EnteredCurrentStatus':{'$gte':time.mktime((2016,1,1,0,0,0,0,0,0)),
                                       '$lt':time.mktime((2017,1,1,0,0,0,0,0,0))},
                                   'LastRemoteHost':{'$regex':'chtc'},
                                  },
                                  projection=['RemoteWallClockTime']):
    avg += row['RemoteWallClockTime']
    num += 1
print('total',avg/3600)

total 137018.11194444445


In [109]:
avg = 0.
num = 0
for row in db.condor_history.find({'EnteredCurrentStatus':{'$gte':time.mktime((2016,1,1,0,0,0,0,0,0)),
                                       '$lt':time.mktime((2017,1,1,0,0,0,0,0,0))},
                                   'LastRemoteHost':{'$regex':'chtc'},
                                  },
                                  projection=['CumulativeSlotTime']):
    avg += row['CumulativeSlotTime']
    num += 1
print('total',avg/3600)

total 7106601.990555556


In [100]:
db.condor_history.find_one({'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$ne':'Comet'},'LastRemoteHost':{'$regex':'comet-..-..\.sdsc\.edu'}})

In [97]:
hosts = defaultdict(float)
for row in db.condor_history.find({'Requestgpus':1,'ExitBySignal':True,'ExitSignal':{'$in':[12,9,15]},'MATCH_EXP_JOBGLIDEIN_ResourceName':'Comet','EnteredCurrentStatus':{'$gte':time.mktime((2017,1,1,0,0,0,0,0,0)),'$lt':time.mktime((2017,1,2,0,0,0,0,0,0))}},projection=['LastRemoteHost','RemoteWallClockTime']):
    hosts[row['LastRemoteHost'].split('@')[-1]] += row['RemoteWallClockTime']/3600
pprint(hosts)
print(sum(hosts.values()))

defaultdict(<class 'float'>,
            {'comet-30-05.sdsc.edu': 381.10416666666737,
             'comet-30-10.sdsc.edu': 1867.5530555555533,
             'comet-31-08.sdsc.edu': 2384.9333333333293,
             'comet-31-10.sdsc.edu': 943.826388888889,
             'comet-31-17.sdsc.edu': 1315.156388888891})
6892.57333333333


In [32]:
start_time = time.time()
pipeline = [
    {'$match':{'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$exists': True},
               'LastRemoteHost': {'$regex':'glidein_'} } },
    {'$group':{'_id':'$MATCH_EXP_JOBGLIDEIN_ResourceName','count':{'$sum':1}}},
]
groups = {row['_id']:row['count'] for row in db.condor_history.aggregate(pipeline)}
print(time.time()-start_time)
print(len(groups))

1498.047780752182
59


In [33]:
sorted(groups.keys(),key=lambda k:groups[k],reverse=True)

['DESY-ZN',
 'CIT_CMS_T2',
 'Crane',
 'GPGrid',
 'CA-SCINET-T2',
 'SU-OG-CE1',
 'xsede-osg',
 'Nebraska',
 'SU-OG-CE',
 'MWT2',
 'CA-MCGILL-CLUMEQ-T2',
 'HOSTED_STANFORD',
 'wuppertalprod',
 'BNL-ATLAS',
 'Tusker',
 'Syracuse',
 'SPRACE',
 'NWICG_NDCMS',
 'OU_OSCER_ATLAS',
 'UNI-DORTMUND',
 'UCSDT2',
 'Purdue-Hadoop',
 'UConn-OSG',
 'RWTH-Aachen',
 'USCMS-FNAL-WC1',
 'Hyak',
 'SU-OG',
 'BEgrid-ULB-VUB',
 'UCD',
 'AGLT2',
 'Clemson-Palmetto',
 'DESY-HH',
 'NPX',
 'UKI-NORTHGRID-MAN-HEP',
 'NYSGRID_CORNELL_NYS1',
 'FLTECH',
 'NUMEP-OSG',
 'Illinois',
 'GridUNESP_CENTRAL',
 'SWT2_CPB',
 'mainzgrid',
 'osgconnect',
 'UTA_SWT2',
 'Vanderbilt',
 'Sandhills',
 'Ultralight',
 'Kansas',
 'Boston',
 'Indiana',
 'UKI-LT2-QMUL',
 'LIDO_Dortmund',
 'Guillimin',
 'IceCube_DE_Dortmund_udo-ce08',
 'CHTC',
 'mainz',
 'Dourtmund',
 'Comet',
 'T2B_BE_IIHE',
 'DESY']

In [42]:
print(groups['Nebraska']+groups['Crane']+groups['Tusker']+groups['Sandhills'])
print(groups['xsede-osg'])
print(groups['CIT_CMS_T2'])

2686390
899221
1524454


In [35]:
import re
start_time = time.time()
pipeline = [
    {'$match':{'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$exists': True},
               'LastRemoteHost': {'$not': re.compile('glidein_') } } },
    {'$group':{'_id':'$MATCH_EXP_JOBGLIDEIN_ResourceName','count':{'$sum':1}}},
]
groups2 = {row['_id']:row['count'] for row in db.condor_history.aggregate(pipeline)}
print(time.time()-start_time)
print(len(groups2))
sorted(groups2.keys(),key=lambda k:groups2[k],reverse=True)

1404.3809270858765
95


['CHTC',
 'xsede-osg',
 'HEP_WISC',
 'T2B_BE_IIHE',
 'Guillimin',
 'Comet',
 'WIPAC',
 'DESY',
 'MATH_WISC',
 'Hyak',
 'CLOUD_WISC',
 'NPX',
 'mainz',
 'CS_WISC',
 'Syracuse',
 'SU-OG-CE',
 'Bartol',
 'USCMS-FNAL-WC1',
 'MWT2',
 'UCSDT2',
 'osgconnect',
 'Crane',
 'parallel',
 'Nebraska',
 'DESY-ZN',
 'Japan',
 'AGLT2',
 'CA-MCGILL-CLUMEQ-T2',
 'Tusker',
 'CA-SCINET-T2',
 'LIDO_Dortmund',
 'SPRACE',
 'Bridges',
 'Alberta',
 'FLTech',
 'SU-OG',
 'HOSTED_STANFORD',
 'Clemson-Palmetto',
 'gzk9000c012',
 'gzk9000c011',
 'CIT_CMS_T2',
 'gzk9000c004',
 'gzk9000c006',
 'Boston',
 'gzk9000c008',
 'gzk9000c009',
 'PHIDO_Dortmund',
 'gzk9000c005',
 'gzk9000c007',
 'gzk9000c010',
 'Uppsala',
 'DESY-HH',
 'Sandhills',
 'GridUNESP_CENTRAL',
 'BNL-ATLAS',
 'FLTECH',
 'UCD',
 'NWICG_NDCMS',
 'NUMEP-OSG',
 'jasper',
 'UNI-DORTMUND',
 'Utah',
 'GPGrid',
 'SU-OG-CE1',
 'MIT',
 'Purdue-Hadoop',
 'mainzgrid',
 'NYSGRID_CORNELL_NYS1',
 'GZK',
 'UConn-OSG',
 'gzk9000c002',
 'gzk9000c001',
 'UTA_SWT2',
 'WIS

In [58]:
print(groups2['T2B_BE_IIHE']+groups2['BEgrid-ULB-VUB'])
print(groups2['DESY'])
print(groups2['mainz'])
print(groups2['Bartol'])
print(groups2['parallel']+groups2['Alberta']+groups2['jasper'])
print(groups2['Japan'])
print(groups2['CA-MCGILL-CLUMEQ-T2'])
print(groups2['LIDO_Dortmund']+groups2['PHIDO_Dortmund']+groups2['UNI-DORTMUND'])
print(groups2['Uppsala'])
print(groups2['wuppertalprod'])

3399555
2476588
1215670
538520
520533
272381
271050
263246
183
51749
2499
